In [ ]:
import requests
import seaborn as sns
import json
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import pandas as pd

In [ ]:
# Adventure Walk 2024 - 50 km
# base_url = 'https://my1.raceresult.com/266144/RRPublish/data/list?key=cbb56f3829f252176e762b342c83e242&listname=Ergebnislisten%7CErgebnisliste%20-%2050k&page=results&contest=0&r=all&l=0'

# Adventure Walk 2024 - 30 km
# base_url = 'https://my1.raceresult.com/266144/RRPublish/data/list?key=cbb56f3829f252176e762b342c83e242&listname=Ergebnislisten%7CErgebnisliste%20-%2025k&page=results&contest=0&r=all&l=0'

# Dresden Marathon 2024 "Allgemein - Ergebnisliste Männer/Frauen"
base_url = "https://my2.raceresult.com/270281/RRPublish/data/list?key=aaee10feb460178330e3e6495129af10&listname=Ergebnislisten%7CErgebnisliste%20M%C3%A4nner%2FFrauen&page=results&contest=0&r=all&l=0"


response = requests.get(f"{base_url}")

In [ ]:
data = json.loads(response.text)

In [ ]:
# for Dresden Marathon 2024
columns = ["id"] + [field["Label"].lower() for field in data["list"]["Fields"][:-2]]

# challenge = "#1_{DE:AOK-Viertelmarathon (10,55 km)|EN:AOK Quarter marathon (10.55 k)|CZ:čtvrtmaraton (10,55 km)}"
# challenge = "#2_{DE:Halbmarathon|EN:half marathon|CZ:půlmaraton}"
# sex = "#3_Männer"
# sex = "#4_Frauen"

challenge = "#3_{DE:Marathon|EN:marathon|CZ:maratón}"
sex = "#5_Männer"
# sex = "#6_Frauen"

# challenge = "#4_{DE:Sparkassen Zehntelmarathon (4,2 km)|EN:Sparkassen 1/10 marathon (4,2km)|CZ:desátýmaratón (4,2 km)}"

col_time = "zeit"
my_name = "FÖRSTER, Stanley"
df = (
    pd.concat(
        [
            pd.DataFrame(values[: len(columns)], index=columns).T
            for values in data["data"][challenge][sex]
        ]
    )
    .reset_index(drop=True)
    .assign(
        time=lambda df: pd.to_timedelta(df[col_time]),
        seconds=lambda df: df["time"].dt.total_seconds(),
    )
)

In [ ]:
# for Adventure Walk 2024
idx_starter_id = 0
idx_name = 1
idx_time = 2

my_name = "Förster, Stanley"
df = pd.concat([
    pd.DataFrame(values, columns=['id', 'name', 'time', 'misc']) for _, values in list(data['data'].values())[0].items()
]).reset_index(drop=True).assign(time=lambda df: pd.to_timedelta(df['time']), seconds=lambda df: df['time'].dt.total_seconds())

In [ ]:
def seconds_to_mhs(sec: int) -> str:
    return f"{sec // 3600:02.0f}:{(sec % 3600) // 60:02.0f}:{(sec % 3600) % 60:02.0f}"


In [ ]:
with plt.style.context("bmh"):
    sns.histplot(
        data=df,
        x='seconds',
    )

    my_record = {
        "my": records.iloc[0]['seconds'],  # TODO - better go with bib number instead of name
    } if len(records := df.query("name == @my_name")) > 0 else {}

    palette = sns.color_palette("bright")
    for i, (prefix, value) in enumerate(dict({
        "min": df['seconds'].min(),
        "median": df['seconds'].median(),
        "max": df['seconds'].max(),
    } | my_record).items()):
        plt.axvline(
            value,
            label=f"{prefix}: {seconds_to_mhs(value)}",
            color=palette[1+i],
        )
    ax = plt.gca()
    ax.tick_params(axis='x', labelrotation=30)
    ax.xaxis.set_major_formatter(FuncFormatter(func=lambda x, pos: seconds_to_mhs(x)))
    plt.xlabel("Time")
    plt.ylabel(f"Participants (total = {len(df)})")
    plt.title(f"{data['list']['HeadLine1']}")

    plt.legend()
    plt.show()